In [72]:
import os
import base64
from pathlib import Path
from typing import List, Dict, Optional, Union
from dotenv import load_dotenv
import json
from datetime import datetime

# LangChain imports
from langchain_core.messages import HumanMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic

# For image processing
from PIL import Image
import io

print("All required libraries imported successfully!")

All required libraries imported successfully!


In [73]:
# Global Configuration Variables for Image Processing
ENABLE_IMAGE_OPTIMIZATION = False  # Set to False to disable image resizing/optimization
IMAGE_RESIZE_RATIO = 0.5  # Resize ratio (0.5 = 50% of original size, 1.0 = no resize)
MAX_IMAGE_DIMENSION = 1024  # Maximum width or height in pixels
MAX_FILE_SIZE_MB = 4  # Maximum file size in MB before optimization
JPEG_QUALITY = 85  # JPEG compression quality (1-100)

class OCRBenchmark:
    """
    A model-agnostic OCR system using LangChain that can extract data from images
    using different AI models (Gemini, OpenAI GPT-4V, Claude).
    """
    
    def __init__(self, model_name: str = "gemini", temperature: float = 0.1):
        """
        Initialize the OCR Benchmark system.
        
        Args:
            model_name: The model to use ("gemini", "openai", "anthropic")
            temperature: Temperature for model generation
        """
        # Load environment variables
        load_dotenv()
        
        self.model_name = model_name
        self.temperature = temperature
        self.model = None
        
        # Supported image formats
        self.supported_formats = {'.png', '.jpg', '.jpeg', '.webp', '.gif'}
        
        # MIME type mapping for correct image format handling
        self.mime_type_map = {
            '.jpg': 'jpeg',
            '.jpeg': 'jpeg', 
            '.png': 'png',
            '.webp': 'webp',
            '.gif': 'gif'
        }
        
        # Image optimization settings (now using global variables)
        self.enable_optimization = ENABLE_IMAGE_OPTIMIZATION
        self.resize_ratio = IMAGE_RESIZE_RATIO
        self.max_image_size = (MAX_IMAGE_DIMENSION, MAX_IMAGE_DIMENSION)
        self.max_file_size_mb = MAX_FILE_SIZE_MB
        self.jpeg_quality = JPEG_QUALITY
        
        # Try to initialize model, but don't fail if API key is missing
        try:
            self.model = self._initialize_model()
        except ValueError as e:
            print(f"⚠️ Warning: {e}")
            print(f"Model '{model_name}' will be initialized when needed if API key becomes available.")
        
    def _initialize_model(self):
        """Initialize the specified model."""
        if self.model_name.lower() == "gemini":
            api_key = os.getenv("GOOGLE_API_KEY")
            if not api_key or api_key == "your_google_api_key_here":
                raise ValueError("GOOGLE_API_KEY not found or not set. Please set your GOOGLE_API_KEY in the .env file")
            return ChatGoogleGenerativeAI(
                model="gemini-2.5-flash",  # Updated to newer model
                temperature=self.temperature,
                google_api_key=api_key,
                timeout=60  # Set reasonable timeout
            )
            
        elif self.model_name.lower() == "openai":
            api_key = os.getenv("OPENAI_API_KEY")
            if not api_key or api_key == "your_openai_api_key_here":
                raise ValueError("OPENAI_API_KEY not found or not set. Please set your OPENAI_API_KEY in the .env file")
            return ChatOpenAI(
                model="gpt-4o",
                temperature=self.temperature,
                timeout=60
            )
            
        elif self.model_name.lower() == "anthropic":
            return ChatAnthropic(
                model_name="claude-3-5-sonnet-20241022",
                temperature=self.temperature,
                timeout=60,
                stop=None
            )
        else:
            raise ValueError(f"Unsupported model: {self.model_name}")
    
    def _optimize_image(self, image_path: str) -> bytes:
        """Optimize image size and quality for API processing."""
        try:
            # Check if optimization is enabled
            if not self.enable_optimization:
                print(f"🔧 Image optimization disabled - using original file")
                with open(image_path, "rb") as f:
                    return f.read()
            
            # Check file size first
            file_size_mb = os.path.getsize(image_path) / (1024 * 1024)
            print(f"📏 Original file size: {file_size_mb:.2f} MB")
            
            with Image.open(image_path) as img:
                # Convert to RGB if necessary
                if img.mode in ('RGBA', 'P'):
                    img = img.convert('RGB')
                
                original_size = img.size
                print(f"📐 Original dimensions: {original_size[0]}x{original_size[1]}")
                
                # Apply resize ratio if specified
                if self.resize_ratio != 1.0:
                    new_width = int(img.size[0] * self.resize_ratio)
                    new_height = int(img.size[1] * self.resize_ratio)
                    img = img.resize((new_width, new_height), Image.Resampling.LANCZOS)
                    print(f"🔧 Resized by ratio {self.resize_ratio}: {img.size[0]}x{img.size[1]}")
                
                # Additional resize if still too large
                elif (img.size[0] > self.max_image_size[0] or 
                      img.size[1] > self.max_image_size[1] or 
                      file_size_mb > self.max_file_size_mb):
                    
                    img.thumbnail(self.max_image_size, Image.Resampling.LANCZOS)
                    print(f"🔧 Thumbnail resize to: {img.size[0]}x{img.size[1]}")
                
                # Save to bytes with optimized quality
                img_byte_arr = io.BytesIO()
                
                # Choose format and quality based on original
                if Path(image_path).suffix.lower() in ['.jpg', '.jpeg']:
                    img.save(img_byte_arr, format='JPEG', quality=self.jpeg_quality, optimize=True)
                    print(f"💾 JPEG quality: {self.jpeg_quality}")
                else:
                    img.save(img_byte_arr, format='PNG', optimize=True)
                    print(f"💾 PNG optimized")
                
                optimized_data = img_byte_arr.getvalue()
                optimized_size_mb = len(optimized_data) / (1024 * 1024)
                print(f"✅ Final optimized size: {optimized_size_mb:.2f} MB")
                
                return optimized_data
                
        except Exception as e:
            print(f"⚠️ Image optimization failed, using original: {e}")
            # Fallback to original file
            with open(image_path, "rb") as f:
                return f.read()
    
    def _ensure_model_initialized(self):
        """Ensure model is initialized before use."""
        if self.model is None:
            try:
                self.model = self._initialize_model()
            except ValueError as e:
                raise ValueError(f"Cannot initialize model '{self.model_name}': {e}")
    
    def encode_image(self, image_path: str) -> str:
        """Encode image to base64 string."""
        with open(image_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')
    
    def extract_data_from_image(self, image_path: str, prompt: str) -> Dict:
        """
        Extract data from a single image using the specified prompt.
        
        Args:
            image_path: Path to the image file
            prompt: Custom prompt for data extraction
            
        Returns:
            Dictionary containing the extracted data and metadata
        """
        try:
            print(f"🚀 Starting image processing...")
            
            # Ensure model is initialized
            self._ensure_model_initialized()
            
            # Validate image file
            if not os.path.exists(image_path):
                return {
                    "error": f"Image file not found: {image_path}",
                    "success": False
                }
            
            # Check file extension
            file_ext = Path(image_path).suffix.lower()
            if file_ext not in self.supported_formats:
                return {
                    "error": f"Unsupported image format: {file_ext}",
                    "success": False
                }
            
            # Get correct MIME type
            mime_type = self.mime_type_map.get(file_ext, file_ext[1:])
            print(f"🔍 File extension: {file_ext} → MIME type: image/{mime_type}")
            
            # Optimize image
            print(f"🖼️ Optimizing image...")
            image_data = self._optimize_image(image_path)
            
            # Encode to base64
            print(f"📝 Encoding to base64...")
            base64_image = base64.b64encode(image_data).decode()
            base64_size_mb = len(base64_image) / (1024 * 1024)
            print(f"📊 Base64 size: {base64_size_mb:.2f} MB")
            
            # Construct data URL
            data_url = f"data:image/{mime_type};base64,{base64_image}"
            
            # Prepare message
            print(f"🤖 Sending to {self.model_name.upper()}...")
            message = HumanMessage(
                content=[
                    {"type": "text", "text": prompt},
                    {
                        "type": "image_url",
                        "image_url": {"url": data_url}
                    }
                ]
            )
            
            # Get response with timeout handling
            if self.model is not None:
                import time
                start_time = time.time()
                response = self.model.invoke([message])
                end_time = time.time()
                print(f"⏱️ API call completed in {end_time - start_time:.2f} seconds")
            else:
                raise ValueError("Model not initialized")
            
            return {
                "image_path": image_path,
                "model_used": self.model_name,
                "prompt": prompt,
                "extracted_data": response.content,
                "success": True,
                "timestamp": datetime.now().isoformat(),
                "processing_time_seconds": end_time - start_time if 'end_time' in locals() else None
            }
            
        except Exception as e:
            print(f"❌ Error during processing: {str(e)}")
            return {
                "image_path": image_path,
                "error": str(e),
                "success": False,
                "timestamp": datetime.now().isoformat()
            }
    
    def get_images_from_directory(self, directory_path: str) -> List[str]:
        """
        Get all supported image files from a directory.
        
        Args:
            directory_path: Path to the directory containing images
            
        Returns:
            List of image file paths
        """
        directory = Path(directory_path)
        if not directory.exists():
            raise ValueError(f"Directory not found: {directory_path}")
        
        image_files = []
        for file_path in directory.iterdir():
            if file_path.is_file() and file_path.suffix.lower() in self.supported_formats:
                image_files.append(str(file_path))
        
        return sorted(image_files)
    
    def process_directory(self, directory_path: str, prompt: str, 
                         output_file: Optional[str] = None) -> List[Dict]:
        """
        Process all images in a directory with the given prompt.
        
        Args:
            directory_path: Path to directory containing images
            prompt: Custom prompt for data extraction
            output_file: Optional JSON file to save results
            
        Returns:
            List of results for each image
        """
        image_files = self.get_images_from_directory(directory_path)
        
        if not image_files:
            print(f"No supported image files found in {directory_path}")
            return []
        
        print(f"Found {len(image_files)} image(s) to process...")
        
        results = []
        for i, image_path in enumerate(image_files, 1):
            print(f"Processing image {i}/{len(image_files)}: {Path(image_path).name}")
            result = self.extract_data_from_image(image_path, prompt)
            results.append(result)
            
            if result["success"]:
                print(f"✅ Successfully processed {Path(image_path).name}")
            else:
                print(f"❌ Failed to process {Path(image_path).name}: {result.get('error', 'Unknown error')}")
        
        # Save results to file if specified
        if output_file:
            self.save_results(results, output_file)
            print(f"Results saved to {output_file}")
        
        return results
    
    def save_results(self, results: List[Dict], output_file: str):
        """Save results to a JSON file."""
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(results, f, indent=2, ensure_ascii=False)
    
    def change_model(self, model_name: str):
        """Change the model being used."""
        self.model_name = model_name
        self.model = None  # Reset model
        try:
            self.model = self._initialize_model()
            print(f"✅ Model changed to: {model_name}")
        except ValueError as e:
            print(f"⚠️ Warning: Failed to initialize {model_name}: {e}")
            print(f"Model will be initialized when needed if API key becomes available.")
    
    def check_api_keys(self) -> Dict[str, bool]:
        """Check which API keys are available."""
        keys_status = {}
        
        google_key = os.getenv("GOOGLE_API_KEY")
        keys_status["gemini"] = bool(google_key and google_key != "your_google_api_key_here")
        
        openai_key = os.getenv("OPENAI_API_KEY")
        keys_status["openai"] = bool(openai_key and openai_key != "your_openai_api_key_here")
        
        anthropic_key = os.getenv("ANTHROPIC_API_KEY")
        keys_status["anthropic"] = bool(anthropic_key and anthropic_key != "your_anthropic_api_key_here")
        
        return keys_status

print("OCRBenchmark class defined successfully!")
print(f"🔧 Global settings: Optimization={ENABLE_IMAGE_OPTIMIZATION}, Resize ratio={IMAGE_RESIZE_RATIO}")

OCRBenchmark class defined successfully!
🔧 Global settings: Optimization=False, Resize ratio=0.5


In [77]:
# Example usage and demonstration
def create_sample_prompts():
    """Create sample prompts for different OCR tasks."""
    return {
        "general_ocr": "Extract all text from this image and format it nicely. Preserve the structure and layout as much as possible.",
        
        "payslip_extraction": """
        This is a payslip/salary slip image. Please extract the following information in JSON format:
        {
          "employee_name": "",
          "employee_id": "",
          "pay_period": "",
          "gross_salary": "",
          "net_salary": "",
          "deductions": [],
          "company_name": ""
        }
        If any field is not found, use null.
        """,
        
        "structured_data": """
        Extract all structured data from this image and organize it in a clear, readable format. 
        Identify tables, forms, or any structured information and present it clearly.
        """,
        
        "key_value_pairs": """
        Extract all key-value pairs from this image. Present them as:
        Key: Value
        Key: Value
        etc.
        """,
        
        "cash_expense_extraction": """
You are an expert document processing assistant. Your task is to extract structured data from invoice/receipt images and return the information in a specific JSON format.

## Instructions:
1. Analyze the provided image carefully to identify all relevant financial information
2. Extract the data according to the JSON schema provided below
3. If any required field is not visible or unclear in the image, use reasonable defaults or null values
4. For monetary amounts, use numbers without currency symbols
5. For dates, use the format found in the document or convert to YYYY-MM-DD if possible
6. Be precise with calculations - ensure totals match the extracted item details
7. The document may contain text in both English and Bangla (Bengali). For Bangla text, provide English transliteration (not translation) - write the Bangla words using English letters to show how they sound.

## Required JSON Schema:
Return your response as a valid JSON object following this exact structure:

{
  "payment_details": {
    "supplier": "string - Name of the supplier/vendor",
    "payment_account": "string - Account used for payment",
    "payment_date": "string - Date of payment",
    "payment_method": "string - Mode of payment (cash, card, bank transfer, etc.)",
    "ref_no": "string - Reference/invoice number",
    "tags": ["array of relevant tags"]
  },
  "item_details": [
    {
      "category": "string - Expense category",
      "description": "string - Item description (optional)",
      "quantity": "number - Quantity of items",
      "unit_price": "number - Price per unit",
      "total": "number - Total amount for this item"
    },
    {
      "category": "string - Expense category for item 2",
      "description": "string - Item description (optional)",
      "quantity": "number - Quantity of items",
      "unit_price": "number - Price per unit", 
      "total": "number - Total amount for this item"
    }
    // ... additional items as found in the document
  ],
  "attachment": "string - File reference if mentioned",
  "memo": "string - Any additional notes or memo",
  "totals": {
    "sub_total": "number - Subtotal before tax and discount",
    "sales_tax": {
      "is_percentage": "boolean - true if tax is %, false if fixed amount",
      "tax": "number - Tax rate / VAT rate (%)",
      "amount": "number - Calculated tax amount"
    },
    "discount": {
      "is_percentage": "boolean - true if discount is %, false if fixed amount", 
      "discount": "number - Discount rate (%)",
      "calculated_amount": "number - Calculated discount amount"
    },
    "total_amount": "number - Final total amount"
  }
}

## Extraction Guidelines:
- **Supplier**: Look for business name, vendor name, or "Bill To" information
- **Payment Account**: Extract account numbers, card details, or payment method info
- **Payment Date**: Find transaction date, invoice date, or payment date
- **Payment Method**: Identify if paid by cash, card, check, bank transfer, etc.
- **Reference Number**: Look for invoice #, receipt #, transaction ID, or reference number
- **Tags**: Generate relevant tags based on the business type or expense category
- **Item Details**: Extract ALL line items from the document - create a separate object for each item/product/service listed
- **Categories**: Classify expenses (office supplies, travel, meals, equipment, etc.)
- **Calculations**: Verify that item totals sum to subtotal, and final calculations are accurate
- **Language Handling**: If you encounter Bangla/Bengali text, transliterate it into English letters (e.g., "টাকা" becomes "taka", "নাম" becomes "naam"). Do not translate the meaning, just write how the Bangla words sound in English.

## Important Notes:
- Return ONLY the JSON object, no additional text or explanations
- If information is missing, use null for strings and 0 for numbers
- Ensure all numbers are numeric values, not strings
- **Extract every single line item** - the item_details array should contain one object for each product/service listed in the document
- Double-check mathematical accuracy of totals and ensure all item totals sum to the subtotal
- VAT and tax should be considered same.
- If no tax/VAT or discount is present, set the respective amounts to 0

Now please analyze the provided image and extract the data according to this schema.
        """
    }
  
# Create sample prompts
sample_prompts = create_sample_prompts()

print("Sample prompts created:")
for name, prompt in sample_prompts.items():
    print(f"\n📝 {name.upper()}:")
    print(f"   {prompt[:100]}...")

print("\nReady to use! Check the next cells for usage examples.")

Sample prompts created:

📝 GENERAL_OCR:
   Extract all text from this image and format it nicely. Preserve the structure and layout as much as ...

📝 PAYSLIP_EXTRACTION:
   
        This is a payslip/salary slip image. Please extract the following information in JSON forma...

📝 STRUCTURED_DATA:
   
        Extract all structured data from this image and organize it in a clear, readable format. 
 ...

📝 KEY_VALUE_PAIRS:
   
        Extract all key-value pairs from this image. Present them as:
        Key: Value
        Ke...

📝 CASH_EXPENSE_EXTRACTION:
   
You are an expert document processing assistant. Your task is to extract structured data from invoi...

Ready to use! Check the next cells for usage examples.


In [78]:
# Initialize the OCR Benchmark system
# By default, it uses Gemini model

try:
    # Initialize with Gemini (default)
    ocr = OCRBenchmark(model_name="gemini", temperature=0.1)
    print("✅ OCR Benchmark system initialized successfully with Gemini!")
    print(f"Current model: {ocr.model_name}")
    
    # Check if we have images to process
    pay_slip_dir = "/home/tanjim_noor/Work/AI OCR BenchMark/test png"
    if os.path.exists(pay_slip_dir):
        image_files = ocr.get_images_from_directory(pay_slip_dir)
        print(f"Found {len(image_files)} image(s) in Pay Slip directory")
        if image_files:
            print("Sample images:")
            for i, img in enumerate(image_files[:3]):  # Show first 3
                print(f"  {i+1}. {Path(img).name}")
            if len(image_files) > 3:
                print(f"  ... and {len(image_files) - 3} more")
    else:
        print("Pay Slip directory not found")
        
except Exception as e:
    print(f"❌ Error initializing OCR system: {e}")
    print("Please make sure to set your API keys in the .env file")

✅ OCR Benchmark system initialized successfully with Gemini!
Current model: gemini
Found 3 image(s) in Pay Slip directory
Sample images:
  1. IMG_20250826_154131.jpg
  2. IMG_20250826_154308~2.jpg
  3. IMG_20250826_154715_1.jpg


In [80]:
# Example 1: Process a single image
# Let's try to process one image from the Pay Slip directory

try:
    pay_slip_dir = "/home/tanjim_noor/Work/AI OCR BenchMark/test png"
    if os.path.exists(pay_slip_dir):
        image_files = ocr.get_images_from_directory(pay_slip_dir)
        
        if image_files:
            # Process the first image with payslip extraction prompt
            first_image = image_files[2]
            print(f"Processing: {Path(first_image).name}")
            print("Using general OCR prompt...")

            result = ocr.extract_data_from_image(first_image, sample_prompts["cash_expense_extraction"])
            
            if result["success"]:
                print("\n✅ SUCCESS!")
                print(f"Model used: {result['model_used']}")
                print(f"Extracted data:\n{result['extracted_data']}")
            else:
                print(f"\n❌ FAILED: {result.get('error', 'Unknown error')}")
        else:
            print("No images found in the directory")
    else:
        print("Pay Slip directory not found")
        
except Exception as e:
    print(f"Error: {e}")
    print("Make sure your API key is set correctly in the .env file")

Processing: IMG_20250826_154715_1.jpg
Using general OCR prompt...
🚀 Starting image processing...
🔍 File extension: .jpg → MIME type: image/jpeg
🖼️ Optimizing image...
🔧 Image optimization disabled - using original file
📝 Encoding to base64...
📊 Base64 size: 5.48 MB
🤖 Sending to GEMINI...
⏱️ API call completed in 22.39 seconds

✅ SUCCESS!
Model used: gemini
Extracted data:
```json
{
  "payment_details": {
    "supplier": "SHWAPNO",
    "payment_account": "City Bank",
    "payment_date": "2025-06-26",
    "payment_method": "Bank Card",
    "ref_no": "D0612506260221",
    "tags": [
      "Groceries",
      "Supermarket",
      "Food",
      "Household"
    ]
  },
  "item_details": [
    {
      "category": "Cooking Oil",
      "description": "Fresh Soyabean Oil 5 Ltr",
      "quantity": 1.00,
      "unit_price": 922.00,
      "total": 922.00
    },
    {
      "category": "Dairy",
      "description": "ULTRA Sweet Curd 500Gm",
      "quantity": 1.00,
      "unit_price": 150.00,
      "tot

In [ ]:
# Example 2: Process all images in directory (Batch Processing)
# This will process all images in the Pay Slip directory

# Uncomment the following code to process all images:
"""
try:
    pay_slip_dir = "/home/tanjim_noor/Work/AI OCR BenchMark/Pay Slip"
    output_file = "/home/tanjim_noor/Work/AI OCR BenchMark/ocr_results.json"
    
    print("Starting batch processing...")
    print(f"Input directory: {pay_slip_dir}")
    print(f"Output file: {output_file}")
    print("=" * 50)
    
    # Process all images with payslip extraction prompt
    results = ocr.process_directory(
        directory_path=pay_slip_dir,
        prompt=sample_prompts["payslip_extraction"],
        output_file=output_file
    )
    
    # Summary
    successful = sum(1 for r in results if r["success"])
    failed = len(results) - successful
    
    print("=" * 50)
    print(f"📊 PROCESSING SUMMARY:")
    print(f"Total images: {len(results)}")
    print(f"Successful: {successful}")
    print(f"Failed: {failed}")
    print(f"Results saved to: {output_file}")
    
except Exception as e:
    print(f"Error during batch processing: {e}")
"""

print("Batch processing code is commented out.")
print("Uncomment the code above to process all images in the directory.")
print("Note: This will use API calls for each image, so be mindful of costs!")

In [ ]:
# Example 3: Switch between different models
# Demonstrate model-agnostic capabilities

print("🔄 Model Switching Examples")
print("=" * 40)

# Current model
print(f"Current model: {ocr.model_name}")

print("\n📝 Available models:")
print("1. gemini - Google Gemini 1.5 Flash")
print("2. openai - OpenAI GPT-4 Vision")  
print("3. anthropic - Claude 3.5 Sonnet")

print("\n🔧 How to switch models:")
print("ocr.change_model('openai')    # Switch to OpenAI")
print("ocr.change_model('anthropic') # Switch to Claude")
print("ocr.change_model('gemini')    # Switch to Gemini")

print("\n⚠️ Note: Make sure to set the corresponding API keys in .env file")

# Example of switching (commented out)
"""
# Switch to OpenAI
try:
    ocr.change_model('openai')
    # Process an image with OpenAI
    result = ocr.extract_data_from_image(image_path, prompt)
except Exception as e:
    print(f"OpenAI error: {e}")

# Switch to Claude
try:
    ocr.change_model('anthropic')
    # Process an image with Claude
    result = ocr.extract_data_from_image(image_path, prompt)
except Exception as e:
    print(f"Claude error: {e}")
"""

In [ ]:
# Example 4: Custom Prompts and Utilities

def create_custom_prompt(task_type: str, specific_fields: Optional[List[str]] = None) -> str:
    """
    Helper function to create custom prompts for different OCR tasks.
    
    Args:
        task_type: Type of document ('invoice', 'receipt', 'form', 'table', 'general')
        specific_fields: List of specific fields to extract
    """
    base_prompts = {
        'invoice': "Extract invoice information including invoice number, date, vendor, amount, tax, and line items.",
        'receipt': "Extract receipt information including store name, date, items purchased, prices, and total amount.",
        'form': "Extract all form fields and their values from this document.",
        'table': "Extract table data and organize it in a structured format with headers and rows.",
        'general': "Extract all text content from this image, preserving structure and formatting."
    }
    
    prompt = base_prompts.get(task_type, base_prompts['general'])
    
    if specific_fields:
        fields_str = ", ".join(specific_fields)
        prompt += f" Focus specifically on extracting: {fields_str}."
    
    prompt += " Present the information in a clear, structured format."
    return prompt

# Example custom prompts
print("🎯 Custom Prompt Examples")
print("=" * 30)

# Create some custom prompts
custom_prompts = {
    "payslip_detailed": create_custom_prompt("form", [
        "employee name", "employee ID", "salary period", 
        "gross pay", "net pay", "tax deductions", "company name"
    ]),
    
    "receipt_analysis": create_custom_prompt("receipt", [
        "store name", "purchase date", "items", "total amount", "payment method"
    ]),
    
    "table_extraction": create_custom_prompt("table"),
    
    "general_ocr": create_custom_prompt("general")
}

for name, prompt in custom_prompts.items():
    print(f"\n📝 {name.upper()}:")
    print(f"   {prompt}")

print("\n✨ You can use these prompts with:")
print("   result = ocr.extract_data_from_image(image_path, custom_prompts['payslip_detailed'])")

In [ ]:
# Example 5: Benchmarking Multiple Models
# Compare results from different models on the same image

def benchmark_models(image_path: str, prompt: str, models: Optional[List[str]] = None) -> Dict:
    """
    Benchmark multiple models on the same image and prompt.
    
    Args:
        image_path: Path to the image file
        prompt: Prompt to use for extraction
        models: List of models to test (defaults to available models)
        
    Returns:
        Dictionary with results from each model
    """
    if models is None:
        models = ["gemini", "openai", "anthropic"]
    
    benchmark_results = {
        "image_path": image_path,
        "prompt": prompt,
        "timestamp": datetime.now().isoformat(),
        "results": {}
    }
    
    for model_name in models:
        print(f"Testing {model_name}...")
        try:
            # Create new OCR instance for each model
            test_ocr = OCRBenchmark(model_name=model_name, temperature=0.1)
            
            # Check if model was initialized successfully
            if test_ocr.model is None:
                print(f"⚠️ {model_name} - Skipped: API key not available")
                benchmark_results["results"][model_name] = {
                    "success": False,
                    "error": f"API key not available for {model_name}",
                    "model_used": model_name
                }
                continue
            
            result = test_ocr.extract_data_from_image(image_path, prompt)
            benchmark_results["results"][model_name] = result
            
            if result["success"]:
                print(f"✅ {model_name} - Success")
            else:
                print(f"❌ {model_name} - Failed: {result.get('error', 'Unknown error')}")
                
        except Exception as e:
            print(f"❌ {model_name} - Error: {str(e)}")
            benchmark_results["results"][model_name] = {
                "success": False,
                "error": str(e),
                "model_used": model_name
            }
    
    return benchmark_results

# Utility function to compare and analyze results
def analyze_benchmark_results(benchmark_results: Dict):
    """Analyze and display benchmark results."""
    print("\n📊 BENCHMARK ANALYSIS")
    print("=" * 50)
    
    results = benchmark_results["results"]
    successful_models = [model for model, result in results.items() if result.get("success", False)]
    failed_models = [model for model, result in results.items() if not result.get("success", False)]
    
    print(f"Image: {Path(benchmark_results['image_path']).name}")
    print(f"Total models tested: {len(results)}")
    print(f"Successful: {len(successful_models)} - {successful_models}")
    print(f"Failed: {len(failed_models)} - {failed_models}")
    
    if successful_models:
        print("\n📝 Extracted Data Comparison:")
        for model in successful_models:
            print(f"\n🤖 {model.upper()}:")
            extracted_data = results[model]["extracted_data"]
            # Show first 200 characters
            preview = extracted_data[:200] + "..." if len(extracted_data) > 200 else extracted_data
            print(f"   {preview}")

print("🚀 Benchmarking utilities defined!")
print("Use benchmark_models() to compare multiple models on the same image.")

# Example usage (commented out):
"""
# Benchmark all models on a single image
image_path = "/path/to/your/image.png"
prompt = sample_prompts["payslip_extraction"]

benchmark_results = benchmark_models(image_path, prompt)
analyze_benchmark_results(benchmark_results)

# Save benchmark results
with open("benchmark_results.json", "w") as f:
    json.dump(benchmark_results, f, indent=2)
"""

# 🚀 Quick Start Guide

## Setup Instructions

1. **Set up API Keys** - Add your API keys to the `.env` file:
   ```bash
   GOOGLE_API_KEY=your_actual_google_api_key
   OPENAI_API_KEY=your_actual_openai_api_key  
   ANTHROPIC_API_KEY=your_actual_anthropic_api_key
   ```

2. **Initialize the OCR System**:
   ```python
   ocr = OCRBenchmark(model_name="gemini")  # Default: Gemini
   ```

3. **Process a Single Image**:
   ```python
   result = ocr.extract_data_from_image("path/to/image.png", "Extract all text")
   ```

4. **Process All Images in Directory**:
   ```python
   results = ocr.process_directory("path/to/directory", "Extract payslip data")
   ```

## Available Models
- **gemini** - Google Gemini 1.5 Flash (Default)
- **openai** - OpenAI GPT-4 Vision  
- **anthropic** - Claude 3.5 Sonnet

## Features
✅ **Model Agnostic** - Switch between models easily  
✅ **Batch Processing** - Process entire directories  
✅ **Custom Prompts** - Flexible prompt engineering  
✅ **JSON Output** - Save results to files  
✅ **Error Handling** - Robust error management  
✅ **Benchmarking** - Compare model performance  

## Next Steps
1. Set your API keys in `.env`
2. Run the initialization cell
3. Try processing a single image first
4. Experiment with different prompts
5. Compare results across models

In [33]:
# Check API Keys Availability
print("🔑 API Keys Status Check")
print("=" * 30)

# Create a temporary OCR instance to check API keys
temp_ocr = OCRBenchmark()
api_status = temp_ocr.check_api_keys()

print("Available models:")
for model, available in api_status.items():
    status = "✅ Available" if available else "❌ Missing API Key"
    print(f"  {model}: {status}")

print(f"\nRecommended model: {next((model for model, available in api_status.items() if available), 'None available')}")

if not any(api_status.values()):
    print("\n⚠️ No API keys found!")
    print("Please set at least one API key in your .env file:")
    print("  GOOGLE_API_KEY=your_key_here")
    print("  OPENAI_API_KEY=your_key_here") 
    print("  ANTHROPIC_API_KEY=your_key_here")
else:
    print(f"\n✅ {sum(api_status.values())} out of {len(api_status)} models available!")

🔑 API Keys Status Check
Available models:
  gemini: ✅ Available
  openai: ❌ Missing API Key
  anthropic: ❌ Missing API Key

Recommended model: gemini

✅ 1 out of 3 models available!


In [ ]:
# Test JPG file support and MIME type handling
print("🔍 Testing JPG File Support")
print("=" * 40)

# Show supported formats
print("Supported image formats:")
for fmt in sorted(ocr.supported_formats):
    print(f"  {fmt}")

print("\nMIME type mappings:")
for ext, mime in ocr.mime_type_map.items():
    print(f"  {ext} → image/{mime}")

print("\n📁 Checking for JPG files in your directories...")

# Check both directories for JPG files
directories_to_check = [
    "/home/tanjim_noor/Work/AI OCR BenchMark/Pay Slip",
    "/home/tanjim_noor/Work/AI OCR BenchMark/test png"
]

for directory in directories_to_check:
    if os.path.exists(directory):
        print(f"\n📂 {directory}:")
        try:
            all_files = list(Path(directory).iterdir())
            jpg_files = [f for f in all_files if f.suffix.lower() in ['.jpg', '.jpeg']]
            
            if jpg_files:
                print(f"  Found {len(jpg_files)} JPG/JPEG file(s):")
                for jpg_file in jpg_files[:5]:  # Show first 5
                    file_ext = jpg_file.suffix.lower()
                    mime_type = ocr.mime_type_map.get(file_ext, file_ext[1:])
                    print(f"    {jpg_file.name} ({file_ext} → image/{mime_type})")
                if len(jpg_files) > 5:
                    print(f"    ... and {len(jpg_files) - 5} more")
            else:
                print("  No JPG files found")
                
            # Show all supported image files
            supported_images = ocr.get_images_from_directory(directory)
            if supported_images:
                print(f"  Total supported images: {len(supported_images)}")
        except Exception as e:
            print(f"  Error reading directory: {e}")
    else:
        print(f"\n📂 {directory}: Directory not found")

print(f"\n✅ JPG files are fully supported!")
print("The system will automatically convert .jpg extensions to 'image/jpeg' MIME type.")

In [32]:
# Quick MIME type test
print("🧪 Testing MIME Type Mapping")
print("=" * 30)

test_extensions = ['.jpg', '.jpeg', '.png', '.webp', '.gif']
for ext in test_extensions:
    mime_type = ocr.mime_type_map.get(ext, ext[1:])
    print(f"{ext} → image/{mime_type}")

# Test with your specific file
test_file = "/home/tanjim_noor/Work/AI OCR BenchMark/Pay Slip/IMG_20250826_154052.jpg"
if os.path.exists(test_file):
    file_ext = Path(test_file).suffix.lower()
    mime_type = ocr.mime_type_map.get(file_ext, file_ext[1:])
    print(f"\n📁 Your file: {Path(test_file).name}")
    print(f"Extension: {file_ext}")
    print(f"MIME type: image/{mime_type}")
    print(f"Should be: image/jpeg")
    print(f"Mapping works: {'✅' if mime_type == 'jpeg' else '❌'}")
else:
    print(f"\n📁 Test file not found: {test_file}")

🧪 Testing MIME Type Mapping
.jpg → image/jpeg
.jpeg → image/jpeg
.png → image/png
.webp → image/webp
.gif → image/gif

📁 Your file: IMG_20250826_154052.jpg
Extension: .jpg
MIME type: image/jpeg
Should be: image/jpeg
Mapping works: ✅


In [31]:
# Diagnostic: Check image file sizes and dimensions
print("🔍 Image Diagnostics")
print("=" * 40)

pay_slip_dir = "/home/tanjim_noor/Work/AI OCR BenchMark/Pay Slip"
if os.path.exists(pay_slip_dir):
    image_files = ocr.get_images_from_directory(pay_slip_dir)
    
    if image_files:
        print(f"Found {len(image_files)} images. Analyzing first few...")
        
        for i, img_path in enumerate(image_files[:3]):
            print(f"\n📁 {i+1}. {Path(img_path).name}")
            
            # File size
            file_size_mb = os.path.getsize(img_path) / (1024 * 1024)
            print(f"   File size: {file_size_mb:.2f} MB")
            
            # Image dimensions
            try:
                with Image.open(img_path) as img:
                    print(f"   Dimensions: {img.size[0]}x{img.size[1]} pixels")
                    print(f"   Mode: {img.mode}")
                    
                    # Estimate base64 size
                    estimated_base64_mb = file_size_mb * 1.33  # Base64 is ~33% larger
                    print(f"   Estimated base64 size: {estimated_base64_mb:.2f} MB")
                    
                    if file_size_mb > 4:
                        print(f"   ⚠️ Large file - will be optimized")
                    if img.size[0] > 1024 or img.size[1] > 1024:
                        print(f"   ⚠️ Large dimensions - will be resized")
                        
            except Exception as e:
                print(f"   ❌ Error reading image: {e}")
    else:
        print("No images found")
else:
    print("Directory not found")

🔍 Image Diagnostics
Found 25 images. Analyzing first few...

📁 1. IMG_20250826_154052.jpg
   File size: 2.98 MB
   Dimensions: 4624x3472 pixels
   Mode: RGB
   Estimated base64 size: 3.96 MB
   ⚠️ Large dimensions - will be resized

📁 2. IMG_20250826_154131.jpg
   File size: 3.70 MB
   Dimensions: 4624x3472 pixels
   Mode: RGB
   Estimated base64 size: 4.92 MB
   ⚠️ Large dimensions - will be resized

📁 3. IMG_20250826_154154.jpg
   File size: 3.52 MB
   Dimensions: 4624x3472 pixels
   Mode: RGB
   Estimated base64 size: 4.69 MB
   ⚠️ Large dimensions - will be resized


In [ ]:
# Image Optimization Configuration Utility
print("🛠️ Image Optimization Configuration")
print("=" * 50)

def update_global_settings(enable_optimization=None, resize_ratio=None, max_dimension=None, 
                          max_file_size_mb=None, jpeg_quality=None):
    """Update global image optimization settings."""
    global ENABLE_IMAGE_OPTIMIZATION, IMAGE_RESIZE_RATIO, MAX_IMAGE_DIMENSION
    global MAX_FILE_SIZE_MB, JPEG_QUALITY
    
    if enable_optimization is not None:
        ENABLE_IMAGE_OPTIMIZATION = enable_optimization
        print(f"✅ Global optimization: {'Enabled' if enable_optimization else 'Disabled'}")
    
    if resize_ratio is not None:
        if 0.1 <= resize_ratio <= 1.0:
            IMAGE_RESIZE_RATIO = resize_ratio
            print(f"✅ Global resize ratio: {resize_ratio}")
        else:
            print(f"❌ Invalid resize ratio: {resize_ratio} (must be 0.1-1.0)")
    
    if max_dimension is not None:
        MAX_IMAGE_DIMENSION = max_dimension
        print(f"✅ Global max dimension: {max_dimension}px")
    
    if max_file_size_mb is not None:
        MAX_FILE_SIZE_MB = max_file_size_mb
        print(f"✅ Global max file size: {max_file_size_mb}MB")
    
    if jpeg_quality is not None:
        if 1 <= jpeg_quality <= 100:
            JPEG_QUALITY = jpeg_quality
            print(f"✅ Global JPEG quality: {jpeg_quality}")
        else:
            print(f"❌ Invalid JPEG quality: {jpeg_quality} (must be 1-100)")

def show_global_settings():
    """Display current global settings."""
    print("🌐 Current Global Settings:")
    print(f"  ENABLE_IMAGE_OPTIMIZATION = {ENABLE_IMAGE_OPTIMIZATION}")
    print(f"  IMAGE_RESIZE_RATIO = {IMAGE_RESIZE_RATIO}")
    print(f"  MAX_IMAGE_DIMENSION = {MAX_IMAGE_DIMENSION}")
    print(f"  MAX_FILE_SIZE_MB = {MAX_FILE_SIZE_MB}")
    print(f"  JPEG_QUALITY = {JPEG_QUALITY}")

# Show current settings
show_global_settings()

print("\n📖 Usage Examples:")
print("# Disable optimization completely:")
print("update_global_settings(enable_optimization=False)")
print("\n# Resize images to 25% of original size:")
print("update_global_settings(resize_ratio=0.25)")
print("\n# Set maximum dimension to 512px:")
print("update_global_settings(max_dimension=512)")
print("\n# Multiple settings at once:")
print("update_global_settings(enable_optimization=True, resize_ratio=0.3, jpeg_quality=70)")